# Single Substitution Simulation Run Trace

(c) 2020 Tom Röschinger. This work is licensed under a [Creative Commons Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/). All code contained herein is licensed under an [MIT license](https://opensource.org/licenses/MIT).

***

In [1]:
using LinearAlgebra, Jevo, Jedi, Plots, LambertW, Statistics
Jedi.default_gr!()

# Parameters
steps = 1* 10^7
l_0 = 10
N = 100
nu = 0
ϵ = 1
n = 4
emat = ϵ * (ones(4, 4) - Matrix{Float64}(I, 4, 4))
f0 = 100/2N

┌ Info: Precompiling Jevo [49da04e1-bf5b-45dd-833b-50344a2c673a]
└ @ Base loading.jl:1278


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-18145092934344084614\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-16686877227040466900\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-18145092934344084614\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

0.5

In [2]:
# Length scale
l0(ϵ, n, f0=f0) = 1/ϵ * lambertw(2 * ϵ^2 * N * 10 * f0 * (n-1)/n^2* exp(5 * ϵ))
l0(κ, l, ϵ, n, f0=f0) = 1/ϵ * lambertw(2 * ϵ^2 * N * l * f0 * (n-1)/n^2 * exp(5 * ϵ)/(1 + κ))

# Find length cost to set length in equilibrium (derived from genetic load)
#lambda(l_opt) = l0()^2/l_opt^2 * n^2 / (n-1) * 1/ϵ

# Find length cost to set length in equilibrium (derived from genetic load modified by length dynamics)

lambda_new(l_opt, ϵ, n, f0=f0) = l0(ϵ, n, f0)^3 / l_opt^2 * n^2 / (n-1) * 1/2
lambda_new(l_opt,κ, l, ϵ, n, f0=f0) = l0(κ, l, ϵ, n, f0)^3 / l_opt^2 * n^2 / (n-1) * 1/2

#l_opt_static(κ, ϵ, n) = l0(ϵ, n) * sqrt(n^2 / ((n - 1) * ϵ * lambda(10)) * (1+κ))
l_opt_dynamic(κ, ϵ, n, f0=f0) = l0(ϵ, n, f0)^(3/2) * sqrt(n^2 / ((n - 1) * 2 * lambda_new(10, ϵ, n, f0)) * (1+κ))

F = Jevo.fermi_fitness(f0=f0, fl=lambda_new(10, ϵ, n)/(2N*l0(ϵ, n, f0)), epsilon=ϵ);

In [3]:
rho=0
pop = Jevo.mono_pop(N=100, l=l_0)
Jevo.initiate!(pop, emat)
t0 = zeros(2, steps)
for i in 1:steps
    Jevo.bp_substitution!(pop, emat, F)
    if rand() < 0/N
        Jevo.driver_mutation!(pop)
    end
    if rand() < 1/(10*N)
        Jevo.l_substitution!(pop, emat, F)
    end
    # Recover lost sites
    #if length(pop.seqs) < 7
        #Jevo.initiate!(pop, emat)
    #end
    t0[:, i] = [Jevo.get_energy(pop, emat), length(pop.seqs)] 
end

rho=3
pop = Jevo.mono_pop(N=100, l=l_0)
Jevo.initiate!(pop, emat)
t5 = zeros(2, steps)
for i in 1:steps
    Jevo.bp_substitution!(pop, emat, F)
    if rand() < rho/N
        Jevo.driver_mutation!(pop)
    end
    if rand() < rho/(10N)
        Jevo.l_substitution!(pop, emat, F)
    end
    # Recover lost sites
    if length(pop.seqs) < 7
        Jevo.initiate!(pop, emat)
    end
    t5[:, i] = [Jevo.get_energy(pop, emat), length(pop.seqs)] 
end

In [4]:
plot(t0[2, 1:100:end], label=0)
plot!(t5[2, 1:100:end], label=rho)

In [5]:
mean(t5[2, 1:100:end])

14.46135